In [22]:
import pandas as pd
colnames=['video_id', 'middle_frame_timestamp', 'x1', 'x2', 'y1', 'y2', 'action_id', 'person_id'] 
kinetics_train = pd.read_csv('/data1/hongn/ava_kinetics_v1_0/kinetics_train_v1.0.csv', names=colnames, header=None)

In [17]:
len(kinetics_train['video_id'].unique())

136842

## AVA

In [1]:
import pandas as pd
colnames=['video_id', 'middle_frame_timestamp', 'x1', 'x2', 'y1', 'y2', 'action_id', 'person_id'] 
val_ava = pd.read_csv('/data1/hongn/ava_kinetics_v1_0/ava_val_v2.2.csv', names=colnames, header=None).dropna()
val_ava.head()

ModuleNotFoundError: No module named 'pandas'

In [1]:
from timesformer.datasets.ava import Ava
# from timesformer.datasets.kinetics import Kinetics
from timesformer.utils.parser import load_config
import torch

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--cfg_file', help="restore checkpoint")
parser.add_argument('--opts', help="restore checkpoint")
args = parser.parse_args(['--cfg_file', "/data2/hongn/TimePSFormer/configs/Kinetics/Moose_divST_8x32_224.yaml"]) 
cfg = load_config(args)

dataset = Ava(cfg, "train")
# kinetic = Kinetics(cfg, "train")

/data2/hongn/miniconda3/envs/moose/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
imgs, label_arrs, idx, _, extra_data = dataset[0]

In [2]:
cfg.TRAIN.BATCH_SIZE

2

In [3]:
from timesformer.datasets import utils as utils
split = "train"
from torch.utils.data._utils.collate import default_collate
import itertools
import numpy as np

def multiple_samples_collate(batch, fold=False):
    """
    Collate function for repeated augmentation. Each instance in the batch has
    more than one sample.
    Args:
        batch (tuple or list): data batch to collate.
    Returns:
        (tuple): collated data batch.
    """
    inputs, labels, video_idx, time, extra_data = zip(*batch)
    inputs = [item for sublist in inputs for item in sublist]
    labels = [item for sublist in labels for item in sublist]
    video_idx = [item for sublist in video_idx for item in sublist]
    time = [item for sublist in time for item in sublist]

    inputs, labels, video_idx, time, extra_data = (
        default_collate(inputs),
        default_collate(labels),
        default_collate(video_idx),
        default_collate(time),
        default_collate(extra_data),
    )
    if fold:
        return [inputs], labels, video_idx, time, extra_data
    else:
        return inputs, labels, video_idx, time, extra_data


def detection_collate(batch):
    """
    Collate function for detection task. Concatanate bboxes, labels and
    metadata from different samples in the first dimension instead of
    stacking them to have a batch-size dimension.
    Args:
        batch (tuple or list): data batch to collate.
    Returns:
        (tuple): collated detection data batch.
    """
    inputs, labels, video_idx, time, extra_data = zip(*batch)
    inputs, video_idx = default_collate(inputs), default_collate(video_idx)
    time = default_collate(time)
    labels = torch.tensor(np.concatenate(labels, axis=0)).float()

    collated_extra_data = {}
    for key in extra_data[0].keys():
        data = [d[key] for d in extra_data]
        if key == "boxes" or key == "ori_boxes":
            # Append idx info to the bboxes before concatenating them.
            bboxes = [
                np.concatenate(
                    [np.full((data[i].shape[0], 1), float(i)), data[i]], axis=1
                )
                for i in range(len(data))
            ]
            bboxes = np.concatenate(bboxes, axis=0)
            collated_extra_data[key] = torch.tensor(bboxes).float()
        elif key == "metadata":
            collated_extra_data[key] = torch.tensor(list(itertools.chain(*data))).view(
                -1, 2
            )
        else:
            collated_extra_data[key] = default_collate(data)

    return inputs, labels, video_idx, time, collated_extra_data

assert split in ["train", "val", "test"]
if split in ["train"]:
    dataset_name = cfg.TRAIN.DATASET
    batch_size = int(cfg.TRAIN.BATCH_SIZE / max(1, cfg.NUM_GPUS))
    shuffle = True
    drop_last = True
elif split in ["val"]:
    dataset_name = cfg.TRAIN.DATASET
    batch_size = int(cfg.TRAIN.BATCH_SIZE / max(1, cfg.NUM_GPUS))
    shuffle = False
    drop_last = False
elif split in ["test"]:
    dataset_name = cfg.TEST.DATASET
    batch_size = int(cfg.TEST.BATCH_SIZE / max(1, cfg.NUM_GPUS))
    shuffle = False
    drop_last = False
    
loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=(False if None else shuffle),
            sampler=None,
            num_workers=cfg.DATA_LOADER.NUM_WORKERS,
            pin_memory=cfg.DATA_LOADER.PIN_MEMORY,
            drop_last=drop_last,
            collate_fn=detection_collate,
            worker_init_fn=utils.loader_worker_init_fn(dataset),
        )

In [4]:
for cur_iter, (inputs, labels, _, _) in enumerate(loader):
    break

KeyboardInterrupt: 

In [12]:
imgs, label_arrs, idx, _, extra_data = next(iter(loader))

In [16]:
len(label_arrs)

4

In [26]:
import torch
isinstance(kinetic, torch.utils.data.IterableDataset)

False

In [21]:
extra_data['boxes'].shape

(6, 4)

In [22]:
label_arrs.shape

(6, 400)

In [17]:
imgs[0].shape

torch.Size([3, 9, 224, 224])

In [12]:
imgs[1].shape

IndexError: list index out of range

## SSv2 Everything

In [20]:
import glob
import os
ssv2_dir = glob.glob('/data1/hongn/SSv2/frames/*')
list_empty_folder = []
for path in ssv2_dir:
    if len(os.listdir(path)) == 0:
        list_empty_folder.append(path)

In [ ]:
['/data1/hongn/SSv2/frames/123178', '/data1/hongn/SSv2/frames/123156']
list_empty_folder

['/data1/hongn/SSv2/frames/123178', '/data1/hongn/SSv2/frames/123156']

In [66]:
#TimeSFormer AVA lavel, turn out, it does not need to be processed
import pandas as pd
val = pd.read_csv('/data1/hongn/SSv2/labels/val.csv')
col_merge = 'original_vido_id video_id frame_id path labels'
val[col_merge.split(' ')] = val[col_merge].str.split(' ', expand=True)
val = val.drop(col_merge, axis=1)
# val.to_csv('/data1/hongn/SSv2/labels/train.csv', index=False, header=True)
# val.head()

In [67]:
from tqdm import tqdm

list_empty_folder = []

for name in tqdm(val.path):
    if(os.path.isfile('/data1/hongn/SSv2/frames/' + name) == False):
        list_empty_folder.append(name)

100%|██████████| 3018451/3018451 [01:19<00:00, 37919.14it/s] 


In [ ]:
list_empty_folder

In [68]:
folder_redo = [name.split('/')[0] for name in list_empty_folder]
set(folder_redo)

set()

## HAA500

In [1]:
import pandas as pd
train = pd.read_csv('/data1/hongn/kinetics-dataset/k400/train.csv')
train.head()

,/data1/hongn/kinetics-dataset/k400/train/8mbN0ubh0q8_000000_000010.mp4,101
0,/data1/hongn/kinetics-dataset/k400/train/9bfc-...,17
1,/data1/hongn/kinetics-dataset/k400/train/VV0KA...,167
2,/data1/hongn/kinetics-dataset/k400/train/sWCnx...,318
3,/data1/hongn/kinetics-dataset/k400/train/Am6c9...,356
4,/data1/hongn/kinetics-dataset/k400/train/pC1hX...,231


In [3]:
train_haa500 = pd.read_csv('/data1/hongn/haa500/raw/climb_icecliff.txt')
train_haa500.head()

,https://www.youtube.com/watch?v=Uo_sDYOvW98,113.19,116.11,1,1.1
0,https://www.youtube.com/watch?v=79s5BD0301o,348.50,351.50,1,1
1,https://www.youtube.com/watch?v=1syox_mjE6s,124.41,127.13,0,1
2,https://www.youtube.com/watch?v=OnV0XbwSAAM,423.48,426.39,0,2
3,https://www.youtube.com/watch?v=Xp83UFuKxyE,616.31,619.07,1,1
4,https://www.youtube.com/watch?v=-iDBPhSc2mU,130.50,133.15,0,1


In [85]:
from glob import glob
import numpy as np

action_folders = glob('/data1/hongn/haa500/video/*')
label_nameidx_map = pd.DataFrame(columns=['label_name', 'label_index'])
# all_data = pd.DataFrame(columns=['paths', 'labels'])
# actions
action_paths = []
labels = []
for i in range(len(action_folders)):
    action_name = action_folders[i].split('/')[-1]
    # print(action_name)
    label_nameidx_map.loc[i] = [action_name, i]
    actioni_paths = glob(f'/data1/hongn/haa500/video/{action_name}/*')
    action_paths.append(actioni_paths)
    labels.append(np.ones(len(actioni_paths)) * i)

action_paths = np.array(action_paths).flatten()
labels = np.array(labels).flatten().astype(int)

In [86]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test = train_test_split(action_paths, labels, test_size=0.33, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.33, random_state=42, shuffle=True)

In [95]:
train = pd.DataFrame(np.stack([X_train, y_train], axis=1), columns=['paths', 'labels'])
val = pd.DataFrame(np.stack([X_val, y_val], axis=1), columns=['paths', 'labels'])
test = pd.DataFrame(np.stack([X_test, y_test], axis=1), columns=['paths', 'labels'])

label_nameidx_map.to_csv('/data1/hongn/haa500/label_nameidx_map.csv',index=False, header=False)
train.to_csv('/data1/hongn/haa500/train.csv',index=False, header=False)
val.to_csv('/data1/hongn/haa500/val.csv',index=False, header=False)
test.to_csv('/data1/hongn/haa500/test.csv',index=False, header=False)

In [90]:
np.stack([X_train, y_train], axis=1)

array([['/data1/hongn/haa500/video/basketball_shoot/basketball_shoot_007.mp4',
        '121'],
       ['/data1/hongn/haa500/video/washing_dishes/washing_dishes_000.mp4',
        '297'],
       ['/data1/hongn/haa500/video/snowboard_jump/snowboard_jump_010.mp4',
        '159'],
       ...,
       ['/data1/hongn/haa500/video/one_arm_push_up/one_arm_push_up_015.mp4',
        '155'],
       ['/data1/hongn/haa500/video/sprint_run/sprint_run_006.mp4', '395'],
       ['/data1/hongn/haa500/video/frisbee_catch/frisbee_catch_014.mp4',
        '432']], dtype='<U109')

In [88]:
y_train

array([121, 297, 159, ..., 155, 395, 432])

In [81]:
labels.astype(int)

array([  0,   0,   0, ..., 499, 499, 499])

In [69]:
import numpy as np
actioni = glob('/data1/hongn/haa500/video/gym_plank/*')
len(actioni)
np.ones(len(actioni)) * i

array([499., 499., 499., 499., 499., 499., 499., 499., 499., 499., 499.,
       499., 499., 499., 499., 499., 499., 499., 499., 499.])

In [68]:
all_data = pd.DataFrame(columns=['paths', 'labels'])
temp = np.stack([actioni, np.ones(len(actioni))], axis=1)
all_data = pd.concat([all_data,pd.Series(temp)], ignore_index=True)
all_data

ValueError: Data must be 1-dimensional, got ndarray of shape (20, 2) instead

In [7]:
actions[12]

'/data1/hongn/haa500/raw/play_ukulele.txt'